```
--- Day 8: Two-Factor Authentication ---

You come across a door implementing what you can only assume is an implementation of two-factor authentication after a long game of requirements telephone.

To get past the door, you first swipe a keycard (no problem; there was one on a nearby desk). Then, it displays a code on a little screen, and you type that code on a keypad. Then, presumably, the door unlocks.

Unfortunately, the screen has been smashed. After a few minutes, you've taken everything apart and figured out how it works. Now you just have to work out what the screen would have displayed.

The magnetic strip on the card you swiped encodes a series of instructions for the screen; these instructions are your puzzle input. The screen is 50 pixels wide and 6 pixels tall, all of which start off, and is capable of three somewhat peculiar operations:

rect AxB turns on all of the pixels in a rectangle at the top-left of the screen which is A wide and B tall.
rotate row y=A by B shifts all of the pixels in row A (0 is the top row) right by B pixels. Pixels that would fall off the right end appear at the left end of the row.
rotate column x=A by B shifts all of the pixels in column A (0 is the left column) down by B pixels. Pixels that would fall off the bottom appear at the top of the column.
For example, here is a simple sequence on a smaller screen:

rect 3x2 creates a small rectangle in the top-left corner:

###....
###....
.......
rotate column x=1 by 1 rotates the second column down by one pixel:

#.#....
###....
.#.....
rotate row y=0 by 4 rotates the top row right by four pixels:

....#.#
###....
.#.....
rotate column x=1 by 1 again rotates the second column down by one pixel, causing the bottom pixel to wrap back to the top:

.#..#.#
#.#....
.#.....
As you can see, this display technology is extremely powerful, and will soon dominate the tiny-code-displaying-screen market. That's what the advertisement on the back of the display tries to convince you, anyway.

There seems to be an intermediate check of the voltage used by the display: after you swipe your card, if the screen did work, how many pixels should be lit?

```

In [55]:
import numpy as np
import re

class Screen(object):
    
    RE_RECT = re.compile("rect ([0-9]+)x([0-9]+)")
    RE_RCOLUMN = re.compile("rotate column x=([0-9]+) by ([0-9]+)")
    RE_RROW = re.compile("rotate row y=([0-9]+) by ([0-9]+)")
    
    def __init__(self, width=50, height=6):
        self.width = 50
        self.height = 6
        self.screen = np.zeros((height, width), dtype=bool)
        
    def rect(self, width, height):
        self.screen[:height, :width] = True
        
    def rotate_column(self, idx, steps):
        self.screen[:,idx] = np.roll(self.screen[:,idx], steps)
        
    def rotate_row(self, idx, steps):
        self.screen[idx,:] = np.roll(self.screen[idx,:], steps)
        
    def execute(self, instruction):
        instruction = instruction.strip()
        
        m = self.RE_RECT.match(instruction)
        if m is not None:
            width, height = m.groups()
            self.rect(int(width), int(height))
            return
        
        m = self.RE_RCOLUMN.match(instruction)
        if m is not None:
            idx, steps = m.groups()
            self.rotate_column(int(idx), int(steps))
            return
        
        m = self.RE_RROW.match(instruction)
        if m is not None:
            idx, steps = m.groups()
            self.rotate_row(int(idx), int(steps))
            return
        
        print("ERROR: unknown instruction '{}'".format(instruction))
            
    def show(self):
        for row in self.screen:
            print("".join(['#' if pixel else '.' for pixel in row]))   
            
    def count(self):
        return self.screen.sum()

In [56]:
example = ['rect 3x2', 'rotate column x=1 by 1', 'rotate row y=0 by 4', 'rotate column x=1 by 1']

In [59]:
# Text example

s = Screen(7, 3)
for ins in example:
    print("\n", ins)
    s.execute(ins)
    s.show()
    
print("\nTotal pixels lit = {}".format(s.count()))


 rect 3x2
###....
###....
.......

 rotate column x=1 by 1
#.#....
###....
.#.....

 rotate row y=0 by 4
....#.#
###....
.#.....

 rotate column x=1 by 1
.#..#.#
#.#....
.#.....

Total pixels lit = 6


In [61]:
with open('inputs/day8.txt', 'rt') as fd:
    s = Screen(50, 6)
    for ins in fd:
        s.execute(ins)
    s.show()
    print("\nTotal pixels lit = ", s.count())

####.####.####.#...##..#.####.###..####..###...##.
#....#....#....#...##.#..#....#..#.#......#.....#.
###..###..###...#.#.##...###..#..#.###....#.....#.
#....#....#......#..#.#..#....###..#......#.....#.
#....#....#......#..#.#..#....#.#..#......#..#..#.
####.#....####...#..#..#.#....#..#.#.....###..##..

Total pixels lit =  115


```
--- Part Two ---

You notice that the screen is only capable of displaying capital letters; in the font it uses, each letter is 5 pixels wide and 6 tall.

After you swipe your card, what code is the screen trying to display?
```

In [62]:
# EFEYKFRFIJ